In [2]:
# PASO 1: CONFIGURACION DE RUTA DE ARCHIVOS

import os

# 1. Definimos las rutas de tu proyecto

FOLDER_RAW = 'Data_Raw/' #Carpeta de los documentos descargados a estandarizar 
FOLDER_PROCESSED = 'Data_Processed/' # Carpeta donde se guardan los documentos procesados
FOLDER_FINAL = 'Datos_Limpios/' #Carpeta final del compilado de documentos

# 2. Verificar los archivos dentro de la carpeta Raw
archivos_raw = [f for f in os.listdir(FOLDER_RAW) if f.endswith('.csv')]

if len(archivos_raw) == 0:
    print("\n⚠️  ¡Atención! No encontré ningún archivo .csv en Data_Raw.")
else:
    print(f"\n🚀 Listo para procesar {len(archivos_raw)} archivo(s):")
    for a in archivos_raw:
        print(f"   - {a}")


🚀 Listo para procesar 31 archivo(s):
   - Chelsea-Hudson Yards (MN0401).csv
   - East Harlem (North) (MN1102).csv
   - East Harlem (South) (MN1101).csv
   - East Midtown-Turtle Bay (MN0604).csv
   - East Village (MN0303).csv
   - Financial District-Battery Park City (MN0101).csv
   - Gramercy (MN0602).csv
   - Greenwich Village (MN0202).csv
   - Hamilton Heights-Sugar Hill (MN0903).csv
   - Harlem (North) (MN1002).csv
   - Harlem (South) (MN1001).csv
   - Hell's Kitchen (MN0402).csv
   - Inwood (MN1203).csv
   - Lower East Side (MN0302).csv
   - Manhattanville-West Harlem (MN0902).csv
   - Midtown South-Flatiron-Union Square (MN0501).csv
   - Midtown-Times Square (MN0502).csv
   - Morningside Heights (MN0901).csv
   - Murray Hill-Kips Bay (MN0603).csv
   - SoHo-Little Italy-Hudson Square (MN0201).csv
   - Stuyvesant Town-Peter Cooper Village (MN0601).csv
   - Tribeca-Civic Center (MN0102).csv
   - Upper East Side-Carnegie Hill (MN0802).csv
   - Upper East Side-Lenox Hill-Roosevelt Isl

In [ ]:
# PASO 2: EXTRACCIÓN Y CONSOLIDACIÓN DE VARIABLES (Sex and Age)

import pandas as pd
import csv
import os
import re
import glob

# 1. Configuración de rutas
ruta_entrada = 'Data_Raw' # Carpeta donde se encuentran todos los archivos
archivos = glob.glob(os.path.join(ruta_entrada, "*.csv"))

# Definimos las etiquetas de edad que queremos extraer
age_labels = [
    "Under 5 years", "5 to 9 years", "10 to 14 years", "15 to 19 years",
    "20 to 24 years", "25 to 29 years", "30 to 34 years", "35 to 39 years",
    "40 to 44 years", "45 to 49 years", "50 to 54 years", "55 to 59 years",
    "60 to 64 years", "65 to 69 years", "70 to 74 years", "75 to 79 years",
    "80 to 84 years", "85 years and over"
]

def procesar_manhattan_sex_age(lista_archivos):
    datos_totales = []
    
    for ruta in lista_archivos:
        try:
            with open(ruta, 'r', encoding='utf-8') as f:
                reader = csv.reader(f)
                filas = list(reader)
                
                # Extraer y separar Nombre y Código de la Zona (Fila 5, Columna 7)
                full_area = filas[4][6]
                match = re.search(r'^(.*)\s\((.*)\)$', full_area)
                nombre_z = match.group(1).strip() if match else full_area
                codigo_z = match.group(2).strip() if match else "N/A"
                
                # Diccionario para esta zona
                registro = {'Zona_Nombre': nombre_z, 'Zona_Codigo': codigo_z}
                
                # Escaneo de datos
                for row in filas:
                    if not row: continue
                    label = row[0].strip()
                    
                    # Buscamos las demas variables: Total population, Male, Female, Edades y Median Age
                    if label == "Total population" and "Total population" not in registro:
                        registro["Total Poblacion"] = int(row[6].replace(',', ''))
                    elif label == "Male" and "Total Male" not in registro:
                        registro["Total Male"] = int(row[6].replace(',', ''))
                    elif label == "Female" and "Total Female" not in registro:
                        registro["Total Female"] = int(row[6].replace(',', ''))
                    elif label in age_labels and label not in registro:
                        registro[label] = int(row[6].replace(',', ''))
                    elif label == "Median age (years)" and "Media de Edad" not in registro:
                        registro["Media de Edad"] = float(row[6].replace(',', ''))

                datos_totales.append(registro)
        except Exception as e:
            print(f"⚠️ Error en archivo {os.path.basename(ruta)}: {e}")

    # Creación del DataFrame
    df = pd.DataFrame(datos_totales)
    
    # Ordenar columnas
    cols_order = ['Zona_Nombre', 'Zona_Codigo', 'Total Poblacion', 'Total Male', 'Total Female'] + age_labels + ['Media de Edad']
    return df[cols_order]

# 2. Ejecución del proceso
df_sex_age_final = procesar_manhattan_sex_age(archivos)

# 3. Visualización del Resultado
display(df_sex_age_final.head()) # Muestra los primeras 5 zonas

# 4. Guardar la base de datos consolidada
df_sex_age_final.to_csv('Data_Processed/Base_Datos_1_Sex_Age_Manhattan.csv', index=False)

# Mensaje de confirmación final
print("-" * 70)
print(f"✅ PROCESO COMPLETADO EXITOSAMENTE")
print(f"📂 La base de datos se ha guardado en: {archivo_final}")
print(f"📊 Total de registros: {len(df_sex_age_final)} zonas de Manhattan.")
print("-" * 70)


In [ ]:
# PASO 3: EXTRACCIÓN Y CONSOLIDACIÓN DE VARIABLES (Nacionalidades)

import pandas as pd
import csv
import os
import re
import glob

# 1. Configuración de rutas
ruta_entrada = 'Data_Raw'
archivos = glob.glob(os.path.join(ruta_entrada, "*.csv"))

# Definicion de nacionalidades del dataset a extraer
mapeo_nacionalidades = {
    # 1. Hispanic or Latino (4 subgrupos)
    "Mexican": "Hispanic or Latino", 
    "Puerto Rican": "Hispanic or Latino", 
    "Cuban": "Hispanic or Latino", 
    "Dominican (Dominican Republic)": "Hispanic or Latino",
    
    # 2. Central American (7 subgrupos)
    "Costa Rican": "Central American", 
    "Guatemalan": "Central American", 
    "Honduran": "Central American", 
    "Nicaraguan": "Central American",
    "Panamanian": "Central American", 
    "Salvadoran": "Central American",
    "Other Central American": "Central American",
    
    # 3. South American (10 subgrupos)
    "Argentinean": "South American", 
    "Bolivian": "South American", 
    "Chilean": "South American", 
    "Colombian": "South American",
    "Ecuadorian": "South American", 
    "Paraguayan": "South American", 
    "Peruvian": "South American", 
    "Uruguayan": "South American", 
    "Venezuelan": "South American",
    "Other South American": "South American",
    
    # 4. Other Hispanic (3 subgrupos)
    "Spaniard": "Other Hispanic", 
    "Spanish": "Other Hispanic", 
    "Spanish American": "Other Hispanic",
    
    # 5. All other Hispanic or Latino 
    "All other Hispanic or Latino": "All other Hispanic or Latino"
}

def procesar_paso3_final(lista_csv):
    resultados = []
    for ruta in lista_csv:
        try:
            with open(ruta, 'r', encoding='utf-8') as f:
                reader = csv.reader(f)
                filas = list(reader)
                
                # Metadata Zona
                full_area = filas[4][6]
                match = re.search(r'^(.*)\s\((.*)\)$', full_area)
                zn = match.group(1).strip() if match else full_area
                zc = match.group(2).strip() if match else "N/A"

                for fila in filas:
                    if not fila or len(fila) < 7: continue
                    label = fila[0].strip().replace(":", "")
                    val_raw = fila[6].replace(',', '').strip()
                    
                    if val_raw.isdigit() and label in mapeo_nacionalidades:
                        resultados.append({
                            'Zona_Nombre': zn,
                            'Zona_Codigo': zc,
                            'Grupo': mapeo_nacionalidades[label],
                            'Subgrupo': label,
                            'Poblacion': int(val_raw)
                        })
        except Exception as e:
            print(f"Error en {os.path.basename(ruta)}: {e}")
    return pd.DataFrame(resultados)

# 2. Ejecución del proceso
df_nacionalidades = procesar_paso3_final(archivos)

# 3. Verificación de los datos
if not df_nacionalidades.empty:
    print("="*80)
    print("📊 AUDITORÍA DE LA BASE DE DATOS: ORIGEN HISPANO")
    print("="*80)
    
    # 3.1. Métricas Globales
    n_grupos = df_nacionalidades['Grupo'].nunique()
    n_subgrupos = df_nacionalidades['Subgrupo'].nunique()
    n_zonas = df_nacionalidades['Zona_Nombre'].nunique()
    
    print(f"✅ Zonas de Manhattan: {n_zonas}/31")
    print(f"✅ Total de Grupos:     {n_grupos}")
    print(f"✅ Total de Subgrupos:  {n_subgrupos}")
    print("-" * 80)

    # 3.2. Desglose para verificar los grupos y subgrupos
    resumen = df_nacionalidades.groupby('Grupo')['Subgrupo'].unique()
    
    for grupo, subgrupos in resumen.items():
        print(f"📂 GRUPO: {grupo.upper()}")
        print(f"   └─ Subgrupos ({len(subgrupos)}): {', '.join(subgrupos)}")
        print(f"   {'.' * 60}")

    print("\n" + "="*80)
    print("💡 VALIDACIÓN: El grupo 'All other Hispanic or Latino' se ha incluido como independiente.")
    print("="*80)

else:
    print("❌ Error: No hay datos en 'df_nacionalidades'. Ejecuta la celda de procesamiento.")

# 4. Visualización del Resultado
display(df_nacionalidades.head()) # Muestra los primeras 5 zonas


# 5. Guardar la base de datos consolidada
df_nacionalidades.to_csv('Data_Processed/Base_Datos_2_Hispanic_Origin_Manhattan.csv', index=False)
print(f"✅ Archivo guardado con {len(df_nacionalidades)} filas.")
        

In [ ]:
# PASO 4: EXTRACCIÓN Y CONSOLIDACIÓN DE VARIABLES (Household Type)

import pandas as pd
import csv
import os
import re
import glob

# 1. Configuración de rutas
ruta_entrada = 'Data_Raw'
archivos = glob.glob(os.path.join(ruta_entrada, "*.csv"))

# Definicion de variables del dataset a extraer
columnas_hogares = [
    "Family households (families)",
    "Married-couple family",
    "Male householder, no spouse present, family",
    "Female householder, no spouse present, family",
    "Nonfamily households",
    "Households with one or more people under 18 years",
    "Households with one or more people 65 years and over",
    "Average household size",
    "Average family size"
]

def procesar_paso4_columnas(lista_csv):
    datos_totales = []
    
    for ruta in lista_csv:
        try:
            with open(ruta, 'r', encoding='utf-8') as f:
                reader = csv.reader(f)
                filas = list(reader)
                
                # Metadata Zona
                full_area = filas[4][6]
                match = re.search(r'^(.*)\s\((.*)\)$', full_area)
                nombre_z = match.group(1).strip() if match else full_area
                codigo_z = match.group(2).strip() if match else "N/A"
                
                registro = {'Zona_Nombre': nombre_z, 'Zona_Codigo': codigo_z}
                
                # Escaneo de datos para las columnas específicas
                for row in filas:
                    if not row: continue
                    # Limpieza de la etiqueta para asegurar coincidencia
                    label = row[0].strip().replace(":", "")
                    label = " ".join(label.split())
                    
                    if label in columnas_hogares and label not in registro:
                        val_raw = row[6].replace(',', '').strip()
                        if val_raw and val_raw not in ["Estimate", "Number", "-"]:
                            try:
                                # Si es un promedio (size), convertimos a float, si no a int
                                if "Average" in label:
                                    registro[label] = float(val_raw)
                                else:
                                    registro[label] = int(val_raw)
                            except:
                                registro[label] = val_raw

                datos_totales.append(registro)
        except Exception as e:
            print(f"⚠️ Error en archivo {os.path.basename(ruta)}: {e}")

    df = pd.DataFrame(datos_totales)
    
    # Ordenar columnas: Nombre, Código y luego las variables de hogares
    cols_order = ['Zona_Nombre', 'Zona_Codigo'] + [c for c in columnas_hogares if c in df.columns]
    return df[cols_order]

# 2. Ejecución del proceso
df_hogares_final = procesar_paso4_columnas(archivos)

# 3. Visualización del Resultado
if not df_hogares_final.empty:
    ruta_salida = 'Data_Processed/Base_Datos_3_Household_Type_Manhattan.csv'
    df_hogares_final.to_csv(ruta_salida, index=False)
    
    print("-" * 80)
    print(f"✅ Archivo Guardado")
    print(f"📂 Ubicación: {ruta_salida}")
    print(f"📊 Zonas procesadas: {len(df_hogares_final)}")
    print("-" * 80)
    
    display(df_hogares_final.head()) # Muestra los primeras 5 zonas
else:
    print("❌ No se pudieron extraer datos para las columnas especificadas.")

In [ ]:
# PASO 5: EXTRACCIÓN Y CONSOLIDACIÓN DE VARIABLES (Employment Status)

import pandas as pd
import csv
import os
import re
import glob

# 1. Configuración de rutas
ruta_entrada = 'Data_Raw'
archivos = glob.glob(os.path.join(ruta_entrada, "*.csv"))

# Definicion de variables del dataset a extraer
columnas_empleo = [
    "Population 16 years and over",
    "In labor force",
    "Civilian labor force",
    "Employed",
    "Unemployed",
    "Armed Forces",
    "Not in labor force"
]

def procesar_paso5_empleo(lista_csv):
    datos_totales = []
    
    for ruta in lista_csv:
        try:
            with open(ruta, 'r', encoding='utf-8') as f:
                reader = csv.reader(f)
                filas = list(reader)
                
                # Metadata Zona
                full_area = filas[4][6]
                match = re.search(r'^(.*)\s\((.*)\)$', full_area)
                nombre_z = match.group(1).strip() if match else full_area
                zc = match.group(2).strip() if match else "N/A"
                
                registro = {'Zona_Nombre': nombre_z, 'Zona_Codigo': zc}
                
                # Buscamos la sección de Employment Status general
                in_section = False
                for row in filas:
                    if not row: continue
                    label_raw = row[0].strip().replace(":", "")
                    label = " ".join(label_raw.split())
                    
                    if label == "Employment Status":
                        in_section = True
                        continue
                    
                    # Dentro de esta seccion buscamos las variables seleccionadas
                    if in_section:
                        if label in columnas_empleo and label not in registro:
                            val_raw = row[6].replace(',', '').strip()
                            if val_raw.isdigit():
                                registro[label] = int(val_raw)
                        
                        # verificar si acabo la seccion
                        if label == "Commuting to Work":
                            break

                datos_totales.append(registro)
        except Exception as e:
            print(f"⚠️ Error en archivo {os.path.basename(ruta)}: {e}")

    df = pd.DataFrame(datos_totales)
    
    # Ordenamos las columnas según jerarquía
    cols_order = ['Zona_Nombre', 'Zona_Codigo'] + columnas_empleo
    return df[cols_order]

# 2. Ejecución del proceso
df_empleo_final = procesar_paso5_empleo(archivos)

# 3. Visualización del Resultado
if not df_empleo_final.empty:
    ruta_salida = 'Data_Processed/Base_Datos_4_Employment_Status_Manhattan.csv'
    df_empleo_final.to_csv(ruta_salida, index=False)
    
    print("-" * 80)
    print(f"✅ Archivo Guardado")
    print(f"📂 Ubicación: {ruta_salida}")
    print(f"📊 Zonas procesadas: {len(df_empleo_final)}")
    print("-" * 80)
    
    display(df_empleo_final.head()) # Muestra los primeras 5 zonas
else:
    print("❌ Error: No se pudo extraer la jerarquía de empleo.")

In [16]:
# PASO 6: EXTRACCIÓN Y CONSOLIDACIÓN DE VARIABLES (Income and Benefits)

import pandas as pd
import csv
import os
import re
import glob

# 1. Configuración de rutas
ruta_entrada = 'Data_Raw'
archivos = glob.glob(os.path.join(ruta_entrada, "*.csv"))

# Definicion de variables del dataset a extraer
columnas_busqueda = [
    "Total households",
    "Household income of less than $10,000",
    "Less than $10,000",
    "$10,000 to $14,999",
    "$15,000 to $24,999",
    "$25,000 to $34,999",
    "$35,000 to $49,999",
    "$50,000 to $74,999",
    "$75,000 to $99,999",
    "$100,000 to $149,999",
    "$150,000 to $199,999",
    "$200,000 or more",
    "Median household income (dollars)",
    "Mean household income (dollars)"
]

def procesar_paso6_auditoria(lista_csv):
    datos_totales = []
    for ruta in lista_csv:
        try:
            with open(ruta, 'r', encoding='utf-8') as f:
                reader = csv.reader(f)
                filas = list(reader)
                full_area = filas[4][6]
                match = re.search(r'^(.*)\s\((.*)\)$', full_area)
                nombre_z = match.group(1).strip() if match else full_area
                codigo_z = match.group(2).strip() if match else "N/A"
                registro = {'Zona_Nombre': nombre_z, 'Zona_Codigo': codigo_z}
                
                for row in filas:
                    if not row: continue
                    label = " ".join(row[0].strip().replace(":", "").split())
                    if label in columnas_busqueda:
                        target_label = "Less than $10,000" if "Household income of less than $10,000" in label else label #renombrar columna
                        if target_label not in registro:
                            val_raw = row[6].replace(',', '').replace('$', '').strip()
                            if val_raw and val_raw not in ["Estimate", "Number", "-", "(X)"]:
                                try:
                                    registro[target_label] = float(val_raw) if "M" in label else int(float(val_raw))
                                except: continue
                datos_totales.append(registro)
        except Exception as e:
            print(f"⚠️ Error en {os.path.basename(ruta)}: {e}")
    
    df = pd.DataFrame(datos_totales)
    columnas_finales = ["Total households", "Less than $10,000", "$10,000 to $14,999", "$15,000 to $24,999", 
                        "$25,000 to $34,999", "$35,000 to $49,999", "$50,000 to $74,999", "$75,000 to $99,999", 
                        "$100,000 to $149,999", "$150,000 to $199,999", "$200,000 or more", 
                        "Median household income (dollars)", "Mean household income (dollars)"]
    cols_order = ['Zona_Nombre', 'Zona_Codigo'] + [c for c in columnas_finales if c in df.columns]
    return df[cols_order]

# 2. Ejecución del proceso
df_ingresos_final = procesar_paso6_auditoria(archivos)

# 3. Visualización del Resultado
if not df_ingresos_final.empty:
    if not os.path.exists('Data_Processed'): os.makedirs('Data_Processed')
    df_ingresos_final.to_csv('Data_Processed/Base_Datos_5_Income_Benefits_Manhattan.csv', index=False)
    
    # Cálculo de variables obtenidas
    total_columnas = len(df_ingresos_final.columns)
    variables_datos = total_columnas - 2 # Sin incluir zona y codigo
    
    print("="*80)
    print(f"✅ Archivo Guardado")
    print(f"📂 Ubicación: {ruta_salida}")
    print("="*80)
    print(f"✅ Zonas procesadas:   {len(df_ingresos_final)}")
    print(f"✅ Variables de datos: {variables_datos} (de 13 esperadas)")
    print(f"✅ Columnas totales:   {total_columnas}")
    print("-" * 80)
    
    display(df_ingresos_final.head()) # Muestra los primeras 5 zonas
else:
    print("❌ Error en la extracción.")

✅ Archivo Guardado
📂 Ubicación: Data_Processed/Base_Datos_5_Income_Benefits_Manhattan.csv
✅ Zonas procesadas:   31
✅ Variables de datos: 13 (de 13 esperadas)
✅ Columnas totales:   15
--------------------------------------------------------------------------------


,Zona_Nombre,Zona_Codigo,Total households,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median household income (dollars),Mean household income (dollars)
0,Chelsea-Hudson Yards,MN0401,35763,1765,1555,1469,1110,2126,3976,2696,5300,4261,11505,126272.0,208851.0
1,East Harlem (North),MN1102,27881,4459,3524,3115,2048,2712,3926,2289,2695,1548,1565,39957.0,69405.0
2,East Harlem (South),MN1101,25874,3605,3113,2944,2032,2496,3384,1859,1969,1305,3167,42076.0,91582.0
3,East Midtown-Turtle Bay,MN0604,23244,1044,426,1103,653,1126,1330,2026,2787,2790,9959,170196.0,266919.0
4,East Village,MN0303,33691,3887,2400,2592,1296,2232,4321,2584,4080,3381,6918,76136.0,136990.0


In [18]:
# PASO 7: EXTRACCIÓN Y CONSOLIDACIÓN DE VARIABLES (Housing Occupancy)

import pandas as pd
import csv
import os
import re
import glob

# 1. Configuración de rutas
ruta_entrada = 'Data_Raw'
archivos = glob.glob(os.path.join(ruta_entrada, "*.csv"))

# Definicion de variables del dataset a extraer
columnas_vivienda = [
    "Total housing units",
    "Occupied housing units",
    "Vacant housing units",
    "Homeowner vacancy rate",
    "Rental vacancy rate"
]

def procesar_paso7_vivienda(lista_csv):
    datos_totales = []
    
    for ruta in lista_csv:
        try:
            with open(ruta, 'r', encoding='utf-8') as f:
                reader = csv.reader(f)
                filas = list(reader)
                
                # Metadata Zona
                full_area = filas[4][6]
                match = re.search(r'^(.*)\s\((.*)\)$', full_area)
                nombre_z = match.group(1).strip() if match else full_area
                zc = match.group(2).strip() if match else "N/A"
                
                registro = {'Zona_Nombre': nombre_z, 'Zona_Codigo': zc}
                
                for row in filas:
                    if not row: continue
                    label = " ".join(row[0].strip().replace(":", "").split())
                    
                    if label in columnas_vivienda and label not in registro:
                        val_raw = row[6].replace(',', '').strip()
                        if val_raw and val_raw not in ["Estimate", "Number", "-", "(X)"]:
                            try:
                                # Las tasas (rates) suelen ser float (ej: 2.5), el resto int
                                if "rate" in label.lower():
                                    registro[label] = float(val_raw)
                                else:
                                    registro[label] = int(float(val_raw))
                            except:
                                continue
                datos_totales.append(registro)
        except Exception as e:
            print(f"⚠️ Error en {os.path.basename(ruta)}: {e}")

    df = pd.DataFrame(datos_totales)
    cols_order = ['Zona_Nombre', 'Zona_Codigo'] + [c for c in columnas_vivienda if c in df.columns]
    return df[cols_order]

# 2. Ejecución del proceso
df_vivienda_final = procesar_paso7_vivienda(archivos)

# 3. Visualización del Resultado
if not df_vivienda_final.empty:
    if not os.path.exists('Data_Processed'): os.makedirs('Data_Processed')
    ruta_csv = 'Data_Processed/Base_Datos_6_Housing_Occupancy_Manhattan.csv'
    df_vivienda_final.to_csv(ruta_csv, index=False)
    
    variables_datos = len(df_vivienda_final.columns) - 2 #Cantidad de variables extraidas sin incluir Zona y Codigo
    
    print("="*80)
    print(f"✅ Archivo guardado: {ruta_csv}")
    print(f"✅ Zonas procesadas: {len(df_vivienda_final)}")
    print(f"✅ Variables extraídas: {variables_datos} de {len(columnas_vivienda)}")
    print("-" * 80)
    
    display(df_vivienda_final.head())
else:
    print("❌ No se pudieron extraer datos de ocupación.")

✅ Archivo guardado: Data_Processed/Base_Datos_6_Housing_Occupancy_Manhattan.csv
✅ Zonas procesadas: 31
✅ Variables extraídas: 5 de 5
--------------------------------------------------------------------------------


,Zona_Nombre,Zona_Codigo,Total housing units,Occupied housing units,Vacant housing units,Homeowner vacancy rate,Rental vacancy rate
0,Chelsea-Hudson Yards,MN0401,42625,35763,6862,3.6,6.3
1,East Harlem (North),MN1102,30668,27881,2787,4.9,5.1
2,East Harlem (South),MN1101,28997,25874,3123,2.4,3.8
3,East Midtown-Turtle Bay,MN0604,31825,23244,8581,8.1,7.6
4,East Village,MN0303,37020,33691,3329,0.7,3.7


In [23]:
# PASO 8: EXTRACCIÓN Y CONSOLIDACIÓN DE VARIABLES (Gross Rent)

import pandas as pd
import csv
import os
import re
import glob

# 1. Configuración de rutas
ruta_entrada = 'Data_Raw'
archivos = glob.glob(os.path.join(ruta_entrada, "*.csv"))

# Definicion de variables del dataset a extraer
columnas_renta = [
    "Occupied units paying rent",
    "Less than $500",
    "$500 to $999",
    "$1,000 to $1,499",
    "$1,500 to $1,999",
    "$2,000 to $2,499",
    "$2,500 to $2,999",
    "$3,000 or more",
    "Median gross rent (dollars)",
    "Mean gross rent (dollars)",
    "No rent paid"
]

def procesar_paso8_renta_completo(lista_csv):
    datos_totales = []
    
    for ruta in lista_csv:
        try:
            with open(ruta, 'r', encoding='utf-8') as f:
                reader = csv.reader(f)
                filas = list(reader)
                
                # Metadata Zona
                full_area = filas[4][6]
                match = re.search(r'^(.*)\s\((.*)\)$', full_area)
                nombre_z = match.group(1).strip() if match else full_area
                zc = match.group(2).strip() if match else "N/A"
                
                registro = {'Zona_Nombre': nombre_z, 'Zona_Codigo': zc}
                
                # Flag para sección de renta
                in_rent_section = False
                
                for row in filas:
                    if not row: continue
                    label = " ".join(row[0].strip().replace(":", "").split())
                    
                    if "GROSS RENT" in label.upper():
                        in_rent_section = True
                        continue
                    
                    if in_rent_section:
                        if label in columnas_renta and label not in registro:
                            val_raw = row[6].replace(',', '').replace('$', '').strip()
                            if val_raw and val_raw not in ["Estimate", "Number", "-", "(X)"]:
                                try:
                                    # Median y Mean como float, el resto como int
                                    if "gross rent (dollars)" in label.lower():
                                        registro[label] = float(val_raw)
                                    else:
                                        registro[label] = int(float(val_raw))
                                except:
                                    continue
                        
                
                datos_totales.append(registro)
        except Exception as e:
            print(f"⚠️ Error en {os.path.basename(ruta)}: {e}")

    df = pd.DataFrame(datos_totales)
    
    # Ordenar columnas 
    cols_order = ['Zona_Nombre', 'Zona_Codigo'] + [c for c in columnas_renta if c in df.columns]
    return df[cols_order]

# 2. Ejecución del proceso
df_renta_final = procesar_paso8_renta_completo(archivos)

# 3. GVisualización del Resultado 
if not df_renta_final.empty:
    if not os.path.exists('Data_Processed'): os.makedirs('Data_Processed')
    ruta_csv = 'Data_Processed/Base_Datos_7_Gross_Rent_Manhattan.csv'
    df_renta_final.to_csv(ruta_csv, index=False)
    
    variables_datos = len(df_renta_final.columns) - 2
    
    print("="*80)
    print(f"✅ Archivo guardado: {ruta_csv}")
    print(f"✅ Zonas procesadas: {len(df_renta_final)}")
    print(f"✅ Variables extraídas: {variables_datos}")
    print("-" * 80)
    
    display(df_renta_final.head())
else:
    print("❌ No se pudieron extraer los datos de renta.")

✅ Archivo guardado: Data_Processed/Base_Datos_7_Gross_Rent_Manhattan.csv
✅ Zonas procesadas: 31
✅ Variables extraídas: 8
--------------------------------------------------------------------------------


,Zona_Nombre,Zona_Codigo,Occupied units paying rent,Less than $500,$500 to $999,"$1,500 to $1,999","$2,000 to $2,499","$2,500 to $2,999","$3,000 or more",No rent paid
0,Chelsea-Hudson Yards,MN0401,26276,1610,2794,1522,2148,2332,12638,407
1,East Harlem (North),MN1102,24899,4903,4987,3164,3004,1924,1387,479
2,East Harlem (South),MN1101,23726,5068,4749,2809,2706,971,3325,226
3,East Midtown-Turtle Bay,MN0604,12435,137,394,927,1664,2044,6425,497
4,East Village,MN0303,28776,4237,4807,2254,2965,2686,8092,381
